In [1]:
from google.colab import files
from IPython.display import Image
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
cd '/gdrive/My Drive/Colab Notebooks/team_UBUNTU'

/gdrive/My Drive/Colab Notebooks/team_UBUNTU


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

In [4]:
df = pd.read_csv('kidney_disease.csv')

In [5]:
df[['htn','dm','cad','pe','ane']] = df[['htn','dm','cad','pe','ane']].replace(to_replace={'yes':1,'no':0})
df[['rbc','pc']] = df[['rbc','pc']].replace(to_replace={'abnormal':1,'normal':0})
df[['pcc','ba']] = df[['pcc','ba']].replace(to_replace={'present':1,'notpresent':0})
df[['appet']] = df[['appet']].replace(to_replace={'good':1,'poor':0,'no':np.nan})
df['classification'] = df['classification'].replace(to_replace={'ckd':1.0,'ckd\t':1.0,'notckd':0.0,'no':0.0})
df.rename(columns={'classification':'class'},inplace=True)

In [6]:
df['pe'] = df['pe'].replace(to_replace='good',value=0) # Not having pedal edema is good
df['appet'] = df['appet'].replace(to_replace='no',value=0)
df['cad'] = df['cad'].replace(to_replace='\tno',value=0)
df['dm'] = df['dm'].replace(to_replace={'\tno':0,'\tyes':1,' yes':1, '':np.nan})
df.drop('id',axis=1,inplace=True)

In [7]:
df

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,0.0,0.0,0.0,121.0,36.0,1.2,NaN,NaN,15.4,44,7800,5.2,1.0,1.0,0.0,1.0,0.0,0.0,1.0
1,7.0,50.0,1.020,4.0,0.0,NaN,0.0,0.0,0.0,NaN,18.0,0.8,NaN,NaN,11.3,38,6000,NaN,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,62.0,80.0,1.010,2.0,3.0,0.0,0.0,0.0,0.0,423.0,53.0,1.8,NaN,NaN,9.6,31,7500,NaN,0.0,1.0,0.0,0.0,0.0,1.0,1.0
3,48.0,70.0,1.005,4.0,0.0,0.0,1.0,1.0,0.0,117.0,56.0,3.8,111.0,2.5,11.2,32,6700,3.9,1.0,0.0,0.0,0.0,1.0,1.0,1.0
4,51.0,80.0,1.010,2.0,0.0,0.0,0.0,0.0,0.0,106.0,26.0,1.4,NaN,NaN,11.6,35,7300,4.6,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,0.0,0.0,0.0,0.0,140.0,49.0,0.5,150.0,4.9,15.7,47,6700,4.9,0.0,0.0,0.0,1.0,0.0,0.0,0.0
396,42.0,70.0,1.025,0.0,0.0,0.0,0.0,0.0,0.0,75.0,31.0,1.2,141.0,3.5,16.5,54,7800,6.2,0.0,0.0,0.0,1.0,0.0,0.0,0.0
397,12.0,80.0,1.020,0.0,0.0,0.0,0.0,0.0,0.0,100.0,26.0,0.6,137.0,4.4,15.8,49,6600,5.4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
398,17.0,60.0,1.025,0.0,0.0,0.0,0.0,0.0,0.0,114.0,50.0,1.0,135.0,4.9,14.2,51,7200,5.9,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [8]:
df2 = df.dropna(axis=0)
df2['class'].value_counts()

0.0    115
1.0     43
Name: class, dtype: int64

In [9]:
df2

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,class
3,48.0,70.0,1.005,4.0,0.0,0.0,1.0,1.0,0.0,117.0,56.0,3.8,111.0,2.5,11.2,32,6700,3.9,1.0,0.0,0.0,0.0,1.0,1.0,1.0
9,53.0,90.0,1.020,2.0,0.0,1.0,1.0,1.0,0.0,70.0,107.0,7.2,114.0,3.7,9.5,29,12100,3.7,1.0,1.0,0.0,0.0,0.0,1.0,1.0
11,63.0,70.0,1.010,3.0,0.0,1.0,1.0,1.0,0.0,380.0,60.0,2.7,131.0,4.2,10.8,32,4500,3.8,1.0,1.0,0.0,0.0,1.0,0.0,1.0
14,68.0,80.0,1.010,3.0,2.0,0.0,1.0,1.0,1.0,157.0,90.0,4.1,130.0,6.4,5.6,16,11000,2.6,1.0,1.0,1.0,0.0,1.0,0.0,1.0
20,61.0,80.0,1.015,2.0,0.0,1.0,1.0,0.0,0.0,173.0,148.0,3.9,135.0,5.2,7.7,24,9200,3.2,1.0,1.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,55.0,80.0,1.020,0.0,0.0,0.0,0.0,0.0,0.0,140.0,49.0,0.5,150.0,4.9,15.7,47,6700,4.9,0.0,0.0,0.0,1.0,0.0,0.0,0.0
396,42.0,70.0,1.025,0.0,0.0,0.0,0.0,0.0,0.0,75.0,31.0,1.2,141.0,3.5,16.5,54,7800,6.2,0.0,0.0,0.0,1.0,0.0,0.0,0.0
397,12.0,80.0,1.020,0.0,0.0,0.0,0.0,0.0,0.0,100.0,26.0,0.6,137.0,4.4,15.8,49,6600,5.4,0.0,0.0,0.0,1.0,0.0,0.0,0.0
398,17.0,60.0,1.025,0.0,0.0,0.0,0.0,0.0,0.0,114.0,50.0,1.0,135.0,4.9,14.2,51,7200,5.9,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [10]:
df2 = df2.reset_index()

In [11]:
df2 = df2.drop('index',axis = 1)

In [12]:
X = df2[df2.columns.difference(['class'])]
y = df2['class']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [18]:
X_train

,age,al,ane,appet,ba,bgr,bp,bu,cad,dm,hemo,htn,pc,pcc,pcv,pe,pot,rbc,rc,sc,sg,sod,su,wc
31,49.0,3.0,1.0,0.0,0.0,129.0,100.0,158.0,0.0,1.0,8.1,1.0,1.0,0.0,24,1.0,3.2,1.0,3.5,11.8,1.010,122.0,0.0,9600
81,33.0,0.0,0.0,1.0,0.0,128.0,80.0,38.0,0.0,0.0,13.1,0.0,0.0,0.0,45,0.0,3.9,0.0,4.5,0.6,1.025,135.0,0.0,6200
51,42.0,0.0,0.0,1.0,0.0,98.0,80.0,20.0,0.0,0.0,13.9,0.0,0.0,0.0,44,0.0,3.5,0.0,5.5,0.5,1.020,140.0,0.0,8400
139,59.0,0.0,0.0,1.0,0.0,113.0,60.0,23.0,0.0,0.0,15.3,0.0,0.0,0.0,54,0.0,3.5,0.0,4.9,1.1,1.020,139.0,0.0,6500
125,73.0,0.0,0.0,1.0,0.0,118.0,80.0,44.0,0.0,0.0,14.8,0.0,0.0,0.0,45,0.0,3.5,0.0,4.7,0.7,1.025,137.0,0.0,9300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,33.0,0.0,0.0,1.0,0.0,100.0,80.0,37.0,0.0,0.0,16.9,0.0,0.0,0.0,52,0.0,4.0,0.0,6.0,1.2,1.025,142.0,0.0,6700
106,63.0,0.0,0.0,1.0,0.0,130.0,70.0,37.0,0.0,0.0,13.4,0.0,0.0,0.0,41,0.0,5.0,0.0,4.7,0.9,1.025,150.0,0.0,7300
14,56.0,4.0,0.0,1.0,0.0,210.0,70.0,26.0,0.0,0.0,16.1,0.0,0.0,0.0,52,0.0,3.8,1.0,5.6,1.7,1.015,136.0,1.0,12500
92,57.0,0.0,0.0,1.0,0.0,105.0,60.0,49.0,0.0,0.0,15.7,0.0,0.0,0.0,44,0.0,4.7,0.0,6.2,1.2,1.020,150.0,0.0,10400


In [ ]:
age - age
al - albumin
ane - anemia
appet - appetite
ba - bacteria
bgr - blood glucose random
bp - blood pressure
bu - blood urea
cad - coronary artery disease
dm - diabetes mellitus
hemo - haemoglobin
htn - hypertension
pc - pus cell
pcc - pus cell clumps
pcv - packed cell volume
pe - pedal edema // 0 is good
pot - potassium
rbc - red blood cells
rc - red blood cell count
sg - specific gravity
sod - sodium
sc - serum creatinine
su - sugar
classification - class

In [ ]:
Age(numerical) --> age in years
Blood Pressure(numerical) bp in mm/Hg
Specific Gravity(nominal) sg - (1.005,1.010,1.015,1.020,1.025)
Albumin(nominal)al - (0,1,2,3,4,5)
Sugar(nominal) su - (0,1,2,3,4,5)
Red Blood Cells(nominal) rbc - (normal,abnormal)
Pus Cell (nominal)pc - (normal,abnormal)
Pus Cell clumps(nominal)pcc - (present,notpresent)
Bacteria(nominal) ba - (present,notpresent)
Blood Glucose Random(numerical) bgr in mgs/dl
Blood Urea(numerical) bu in mgs/dl
Serum Creatinine(numerical) sc in mgs/dl
Sodium(numerical) sod in mEq/L
Potassium(numerical) pot in mEq/L
Haemoglobin(numerical) hemo in gms
Packed Cell Volume(numerical)
White Blood Cell Count(numerical) wc in cells/cumm
Red Blood Cell Count(numerical) rc in millions/cmm
Hypertension(nominal) htn - (yes,no)
Diabetes Mellitus(nominal) dm - (yes,no)
Coronary Artery Disease(nominal) cad - (yes,no)
Appetite(nominal) ppet - (good,poor)
Pedal Edema(nominal) pe - (yes,no)
Anemia(nominal)ane - (yes,no)
Class (nominal) class - (ckd,notckd)

In [14]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [15]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

1.0

In [ ]:
y_pred

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0.,
       0., 0., 0., 0., 1., 1., 1.])

In [ ]:
y_test

128    0.0
45     0.0
134    0.0
156    0.0
90     0.0
29     1.0
96     0.0
98     0.0
95     0.0
140    0.0
19     1.0
56     0.0
15     1.0
69     0.0
24     1.0
30     1.0
122    0.0
109    0.0
93     0.0
65     0.0
150    0.0
18     1.0
12     1.0
9      1.0
Name: class, dtype: float64

In [33]:
features = []
age = float(input("enter your age"))
features.append(age)
al = float(input("enter your albumin level in the range of 0-very low ,1,2,3,4,5-very high"))
features.append(al)
ane = float(input("are u anemic 1:yes 0:no"))
features.append(ane)
appet = float(input("Do you have appetite 1:yes 0:no"))
features.append(appet)
bact = float(input("Do you have bacterial infection 1:yes 0:no"))
features.append(bact)
bg = float(input("enter you blood glucose level"))
features.append(bg)
bp = float(input("enter yout blood pressure"))
features.append(bp)
bu = float(input("enter your blood urea level"))
features.append(bu)
cad = float(input("Do you have coronary artery disease 1:yes 0:no"))
features.append(cad)
dm = float(input("DO you have diabetes mellitus 1:yes 0:no "))
features.append(dm)
hemp = float(input("enter your hemoglobin level"))
features.append(hemp)
htn = float(input("Do you have hypertension 1:yes 0:no"))
features.append(htn)
pc = float(input("Do you have pusscell 1:yes 0:no"))
features.append(pc)
pcc = float(input("Do you have pusscell clamps 1:yes 0:no"))
features.append(pcc)
pcv = float(input("enter your packed cell volume"))
features.append(pcv)
pe = float(input("Do you have pedal edema 0 is good 1:yes 0:no"))
features.append(pe)
pot = float(input("enter your body potassium level"))
features.append(pot)
rbc = float(input("hows your rbc count 0:normal 1:abnormal "))
features.append(rbc)
rc = float(input("enter your rbc count"))
features.append(rc)
sc = float(input("enter your serum creatinine count"))
features.append(sc)
sg = float(input("enter specifc gravity (1.005,1.010,1.015,1.020,1.025)"))
features.append(sg)
sod = float(input("enter your sodium level"))
features.append(sod)
su = float(input("enter your sugar level 0-very low ,1,2,3,4,5-very high"))
features.append(su)
wc = float(input("enter your white blood count"))
features.append(wc)
#30.0	0.0	0.0	1.0	0.0	82.0	80.0	42.0	0.0	0.0	14.9	0.0	0.0	0.0	45	0.0	5.0	0.0	5.9	0.7	1.025	146.0	0.0	9400

enter your age30
enter your albumin level in the range of 0-very low ,1,2,3,4,5-very high0
are u anemic 1:yes 0:no0
Do you have appetite 1:yes 0:no1
Do you have bacterial infection 1:yes 0:no0
enter you blood glucose level82
enter yout blood pressure80
enter your blood urea level42
Do you have coronary artery disease 1:yes 0:no0
DO you have diabetes mellitus 1:yes 0:no 0
enter your hemoglobin level14.9
Do you have hypertension 1:yes 0:no0
Do you have pusscell 1:yes 0:no0
Do you have pusscell clamps 1:yes 0:no0
enter your packed cell volume45
Do you have pedal edema 0 is good 1:yes 0:no0
enter your body potassium level5
hows your rbc count 0:normal 1:abnormal 0
enter your rbc count5.9
enter your serum creatinine count0.7
enter specifc gravity (1.005,1.010,1.015,1.020,1.025)1.025
enter your sodium level146
enter your sugar level 0-very low ,1,2,3,4,5-very high0
enter your white blood count9400


In [34]:
vocab = X_test.columns.tolist()[:]

In [35]:
y_test_in = pd.DataFrame(features, vocab)
X_eval = y_test_in.T

In [36]:
y_eval = model.predict(X_eval)

In [37]:
y_eval

array([0.])

In [39]:
import pickle
pickle.dump(model, open("kidney.sav", 'wb'))

In [45]:
import joblib
model_kidney = joblib.load('kidney.sav')

In [46]:
y_eval = model_kidney.predict(X_eval)

In [48]:
y_eval

array([0.])